# 숙박 데이터셋 만들기


1. 비짓제주 크롤링 데이터 기준으로 필터링 
2. 트립어드바이저 크롤링 데이터에서 가격정보 추가 
3.숙박시설 현황 데이터에서 객실 수 정보 추가
4. 마이리얼트립 크롤링 데이터에서 가격정보 추가


In [1]:
import pandas as pd
import numpy as np
import re
import requests
import logging
from tqdm import tqdm

In [2]:
def name_preprocessing(name):
    name = name.replace(' (', '(').replace(') ', ')').strip()
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[\(\)㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = re.sub('[\[.\]]','',name)
    return name.strip()

## 1. 비짓제주 크롤링 데이터로 필터링
- 현재 운영 중인 데이터만 반영하기 위하여 비짓제주 크롤링 데이터 기준으로 필터링

In [3]:
vj_craw_lod_org = pd.read_csv('data/VISITJEJU_숙박크롤링.csv') # crawling data
vj_view = pd.read_csv('data/PRE_FINAL_JT_MT_ACCTO_TRRSRT_SCCNT_LIST.csv') # given data
vj_craw_lod = vj_craw_lod_org.copy()

In [4]:
# 원래 있던 기본 데이터에서 숙박만 남김
vj_lod = vj_view.copy()
vj_lod = vj_view[vj_view['CL_NM']=='숙박']
vj_lod = vj_lod.reset_index(drop=True) ; vj_lod

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO
0,1915지오하우스,숙박,['1915 지오하우스'],['제주특별자치도 서귀포시 성산읍 성산등용로 14'],12129,38,43,33,44,40,47
1,521게스트하우스,숙박,['521 게스트 하우스'],['제주특별자치도 제주시 애월읍 중용길 52-1 (신엄리) 521 게스트 하우스'],12129,5,7,12,26,28,48
2,belazybb,숙박,['be Lazy B&B'],['제주특별자치도 서귀포시 남원읍 위미해안로129번길 17'],12129,13,11,18,23,20,23
3,bk호텔제주,숙박,['BK호텔제주'],['제주특별자치도 서귀포시 칠십리로91번길 12'],12129,9,16,15,11,15,13
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],['제주특별자치도 서귀포시 이어도로 450'],12129,11,15,16,21,20,16
...,...,...,...,...,...,...,...,...,...,...,...
978,흰수염고래리조트,숙박,['흰수염고래리조트'],['제주특별자치도 제주시 애월읍 일주서로 6818'],12129,10,28,20,46,64,59
979,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],['제주특별자치도 서귀포시 예래해안로 542'],12129,106,124,135,96,107,101
980,힐링캠프펜션,숙박,['힐링캠프펜션'],['서귀포시 남원읍 위미항구로 96'],12129,12,12,12,21,27,20
981,힐링하우스,숙박,['힐링하우스'],['제주특별자치도 제주시 홍랑길 4-6'],4042,0,0,0,0,0,0


In [5]:
# 크롤링 데이터 이름 preprocessing -> 새로운 열 만듦
vj_craw_lod['name_PRE'] = list(map(lambda x : name_preprocessing(x), vj_craw_lod['name'])) ; vj_craw_lod

,name,stars,tags,address,telephone,like,heart,numreview,view,detail_text,rec_tour,rec_restaurant,rec_lodgment,rec_bigdata,detail_side_box,name_PRE
0,나미송 머무는 곳,5점,#민박 #고향민박 #독채 #정원 #주방기구 #자연경관,제주특별자치도 제주시 애월읍 평화로 2476,(+82) 010-8307-0531,25,27,157,"34,335",애월읍 시인의 마을에 위치한 나미송 머무는 곳 민박은 요즘 찾아보기 힘든 '진짜' ...,"['멍멍플레이스', '장전리 벚꽃축제길', '제주다']","['오메오메', '귤따고 레드향갈고', '무인카페 산책']","['아마빌레', '잇츠힐', '아뜨네통나무펜션']","['제주이야기펜션', '나미송 머무는 곳', '캠핑트리펜션']","{'소개': '자연과 사람의 서정이 함께 흐르는 아름다운 민박집', '이용 시간':...",나미송머무는곳
1,해비치호텔앤드리조트,5점,#휴식/치유 #5성급호텔 #호텔 #숙소 #리조트 #가족호텔 #공공와이파이존 #양식레...,제주특별자치도 서귀포시 표선면 민속해안로 537,(+82) 064-780-8100,14,71,45,"37,467",해가 처음 비추는 곳이라는 뜻의 해비치 호텔&리조트는 에메랄드 빛 바다가 눈앞에 펼...,"['신천리(용궁올레)', '허브동산 미디어 파사드 (media facade)', '...","['오늘은 녹차한잔&쇼핑', '다미진', '뚜르']","['로그빌리지 펜션', '블랑게스트하우스', '엘마르게스트하우스']","['돌토고리족욕카페', '홍삼흑돼지', '제주광어씨푸드레스토랑 표선점']",{'소개': '제주의 푸른 바다와 자연을 만끽할 수 있는 표선에 위치한 호텔&리조트...,해비치호텔앤드리조트
2,신라호텔제주,5점,#최고급 #VIP #중문 #신라 #중문관광단지 #5성급호텔 #숙소 #게스트하우스 #...,제주도 서귀포시 중문관광로 72번길 75,(+82) 064-735-5114,7,64,43,"24,912",중문관광단지에 위치한 제주 신라호텔은 90년 개관 이후 이국적인 분위기와 최고급 시...,"['스위트호텔 아로마', '중문색달해변(해산물)', '중문컨트리클럽']","['내담', '가든이다', '돈이랑 서귀포점']","['올리수펜션', '스머프하우스', '해성펜션']","['스위트호텔 아로마', '스위트호텔 제주', '롯데호텔 브이스파']","{'소개': '파스텔 톤의 인테리어, 세계 유명 작가들의 예술작품과 아열대 정원이 ...",신라호텔제주
3,롯데호텔 제주,5점,#호텔 #5성급 #중문 #리조트 #숙소 #가족호텔 #수영장 #양식레스토랑 #자연경관...,제주특별자치도 서귀포시 중문관광로72번길 35,(+82) 064-731-1000,9,46,38,"24,501",푸른 제주바다와 하늘이 맞닿은 곳에 위치한 롯데호텔 제주는 천국 같은 휴식과 여행의...,"['제주국제평화센터', '천제연폭포 & 제주4.3중문면희생자위령비']","['중문대들보', '중문솥뚜껑', '돈이랑 서귀포점']","['블룸호텔', '켄싱턴리조트 제주중문', '스위트호텔 제주']","['롯데호텔 제주', '김서프제주', '초콜릿랜드']","{'소개': '사계절 온수풀과 이국적인 풍경이 매혹적인 리조트형 호텔', '숙박 업...",롯데호텔제주
4,제주조천스위스마을게스트하우스,5점,#숙박 #숙소 #게스트하우스 #마을관광 #올레길 #식당,제주특별자치도 제주시 조천읍 함와로 566-27,(+82) 064-744-6060,12,49,29,"50,906","조천읍 와산리에 위치한 제주조천스위스마을은 총 4개 단지, 66세대로 구성되어 있는...","['북촌환해장성', '렛츠런팜 양귀비꽃', '불카분낭 (불타버린 나무)']","['바캉스샤브샤브', '아따블르', '금보가든']","['제주와요펜션', '오션그랜드호텔 (함덕)', '하루앤하루']","['세미오름(샘이오름)', '숲골 제주독채펜션', '더 콘테나']","{'소개': '조천읍 와산리에 위치한 농촌 관광 공동체 마을', '이용 시간': '...",제주조천스위스마을게스트하우스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,비앤비호텔,NaN,#숙소 #호텔 #공공와이파이존 #주차장 #교통 #공항 #단체여행객 #발렛파킹 #무장...,제주특별자치도 제주시 삼무로3길 48,(+82) 064-774-8400,0,2,0,"1,305","제주호텔 비앤비는 제주국제공항과는 2.5km, 바오젠거리는 300m. 신라및 롯데...","['관음사 군 주둔지 옛터', '서부두명품횟집거리', '삼양 셋다리물']","['고망딱새', '고객식당', '갯마을']","['골든비치', '호텔더원', '더오라']","['디셈버호텔', '밸류호텔 월드와이드 제주', '캐니언파크']","{'소개': '제주국제공항에서 가까운 편리한 호텔', '이용 시간': 'Checki...",비앤비호텔
935,비오하우스,NaN,#숙소 #펜션 #휴양펜션 #힐링 #해수욕장 #주방기구 #수영장,제주특별자치도 제주시 구좌읍 월정3길 53-5,(+82) 010-5353-7872,0,2,0,"1,541",비오하우스는 에메랄드빛 월정리 해변가 인근에 위치 해 있다. 여행에서 빠질 수 없...,"['다랑쉬굴', '송당리', '뒤굽은이오름']","['말이', '돌코롬허니', '너는파라다이스길리']","['놀멍쉬멍고르멍', '나무아래독채펜션', '비자낭달집']","['비오하우스', '모래비카페', '달빛고운펜션']","{'소개': '비오하우스는 월정리해변 근처에 위치한 펜션입니다.', '이용 시간':...",비오하우스
936,비자낭달집,NaN,#펜션 #숙소 #휴양펜션 #힐링 #힐링쉼터 #오름투어 #제주시다랑쉬오름,제주특별자치도 제주시 구좌읍 비자림로 2279,(+82) 070-8801-2279,0,0,0,938,"천년의 숲 비자림이 걸어서 10분, 평대리해수욕장이 차로 8분 거리에 있어요. 다...","['동복환해장성', '선족이오름(알선족이)', '세인트포골프장']","['레드썬셋', '뵈뵈', '배롱개']","['하도리보통날', '놀멍쉬멍고르멍', '다올제민박']","['질그랭이센터', '카페비몽', '놀놀카페']","{'소개': '천년의 숲 비자림에서 힐링하다', '이용 시간': 'Checkin :...",비자낭달집
937,월정에비뉴,NaN,#휴식/힐링 #액티비티 #숙소 #수영장 #공공와이파이존 #레저/체험,제주특별자치도 제주시 구좌읍 해맞이해안로 486,(+82) 064-784-9006,0,0,0,"1,256",서정적인 풍경의 마을 월정리 해변에 위치한 월정에비뉴는 숙박 / 액티비티 / F&B...,"['김녕항', '뒤굽은이오름', '문석이오름']","['동복리해녀촌', '너는파라다이스길리', '돌코롬허니']","['감비나무', '가비오타 펜션', '다올제민박']","['제주올레 20코스', '뒹글하우스', '토끼문']","{'소개': '에메랄드 빛깔이 아름다운 월정리 해변 앞, 오션뷰와 넓은 인피니티풀을...",월정에비뉴


In [6]:
# 안 쓸 열들 drop
vj_craw_lod = vj_craw_lod.drop(['stars', 'address', 'telephone', 'numreview', 'view', 'rec_tour', 'rec_restaurant', 'rec_lodgment', 'rec_bigdata', 'detail_side_box'], axis=1) ; vj_craw_lod

,name,tags,like,heart,detail_text,name_PRE
0,나미송 머무는 곳,#민박 #고향민박 #독채 #정원 #주방기구 #자연경관,25,27,애월읍 시인의 마을에 위치한 나미송 머무는 곳 민박은 요즘 찾아보기 힘든 '진짜' ...,나미송머무는곳
1,해비치호텔앤드리조트,#휴식/치유 #5성급호텔 #호텔 #숙소 #리조트 #가족호텔 #공공와이파이존 #양식레...,14,71,해가 처음 비추는 곳이라는 뜻의 해비치 호텔&리조트는 에메랄드 빛 바다가 눈앞에 펼...,해비치호텔앤드리조트
2,신라호텔제주,#최고급 #VIP #중문 #신라 #중문관광단지 #5성급호텔 #숙소 #게스트하우스 #...,7,64,중문관광단지에 위치한 제주 신라호텔은 90년 개관 이후 이국적인 분위기와 최고급 시...,신라호텔제주
3,롯데호텔 제주,#호텔 #5성급 #중문 #리조트 #숙소 #가족호텔 #수영장 #양식레스토랑 #자연경관...,9,46,푸른 제주바다와 하늘이 맞닿은 곳에 위치한 롯데호텔 제주는 천국 같은 휴식과 여행의...,롯데호텔제주
4,제주조천스위스마을게스트하우스,#숙박 #숙소 #게스트하우스 #마을관광 #올레길 #식당,12,49,"조천읍 와산리에 위치한 제주조천스위스마을은 총 4개 단지, 66세대로 구성되어 있는...",제주조천스위스마을게스트하우스
...,...,...,...,...,...,...
934,비앤비호텔,#숙소 #호텔 #공공와이파이존 #주차장 #교통 #공항 #단체여행객 #발렛파킹 #무장...,0,2,"제주호텔 비앤비는 제주국제공항과는 2.5km, 바오젠거리는 300m. 신라및 롯데...",비앤비호텔
935,비오하우스,#숙소 #펜션 #휴양펜션 #힐링 #해수욕장 #주방기구 #수영장,0,2,비오하우스는 에메랄드빛 월정리 해변가 인근에 위치 해 있다. 여행에서 빠질 수 없...,비오하우스
936,비자낭달집,#펜션 #숙소 #휴양펜션 #힐링 #힐링쉼터 #오름투어 #제주시다랑쉬오름,0,0,"천년의 숲 비자림이 걸어서 10분, 평대리해수욕장이 차로 8분 거리에 있어요. 다...",비자낭달집
937,월정에비뉴,#휴식/힐링 #액티비티 #숙소 #수영장 #공공와이파이존 #레저/체험,0,0,서정적인 풍경의 마을 월정리 해변에 위치한 월정에비뉴는 숙박 / 액티비티 / F&B...,월정에비뉴


In [7]:
vj_filter_lod = pd.merge(left = vj_lod, right = vj_craw_lod, how = 'inner', left_on = 'AREA_NM_PRE', right_on = 'name_PRE').reset_index(drop=True)

In [8]:
# PREPROCESSING NAME이 두 개인 경우 확인
vj_filter_lod['AREA_NM_PRE'].value_counts()

나바론민박        2
1915지오하우스    1
자연인          1
저스트슬립제주      1
전원민박         1
            ..
발로바쉬         1
발리인제주        1
밧돌게스트하우스     1
방울풍뎅이하우스     1
힐링캠프펜션       1
Name: AREA_NM_PRE, Length: 904, dtype: int64

In [9]:
# 나바론민박이 크롤링 한 데이터에서 나바론 민박, 나바론민박으로 두 개 들어가 있었나봄
# like, heart가 있는 아래 행만 남기고 삭제 해 줄 것
vj_filter_lod[vj_filter_lod['AREA_NM_PRE'] == '나바론민박']

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,name,tags,like,heart,detail_text,name_PRE
89,나바론민박,숙박,['나바론 민박' '나바론민박'],['제주특별자치도 제주시 추자면 추자로 76-1'],24258,35,33,34,55,39,41,나바론 민박,#추자나바론 #바비큐장 #추자도 #숙소 #민박 #농어촌민박 #특산품판매장,0,0,숙소 창 밖으로 보이는 바다뷰 세탁서비스와 바비큐장 및 픽업서비스 이용 가능하다,나바론민박
90,나바론민박,숙박,['나바론 민박' '나바론민박'],['제주특별자치도 제주시 추자면 추자로 76-1'],24258,35,33,34,55,39,41,나바론민박,#민박 #숙소 #농어촌민박 #교통 #조식 #음식 #주차장 #공공와이파이존 #교통 #...,0,1,"아름다운 섬 추자도에 위치한 민박집이다. 제주 앞바다를 품에 안은 아름다움, 내집 ...",나바론민박


In [10]:
vj_filter = vj_filter_lod.drop(89, axis=0).reset_index(drop=True)

In [11]:
# 근데 일단 못 들어간 애들 상태가 궁금해서 한번 확인
checker = pd.merge(left = vj_lod, right = vj_craw_lod, how = 'outer', left_on = 'AREA_NM_PRE', right_on = 'name_PRE').reset_index(drop=True)
# 얘네처럼 들어갈 수 있는데 이름 형식 때문에 안 된 애들이 몇 개 있는 것 같음
# 어차피 최대 32개니까 하나씩 검색 해 가면서 맞춰주려고 함
checker[(checker['name_PRE'] == 'oh세화') | (checker['AREA_NM_PRE'] == 'oh세화오세화게스트하우스')]

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,name,tags,like,heart,detail_text,name_PRE
12,oh세화오세화게스트하우스,숙박,['Oh! 세화! (오세화게스트하우스)'],['제주특별자치도 제주시 구좌읍 세송로 8-11'],12129.0,43.0,71.0,74.0,106.0,99.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oh! 세화!,#부모 #커플 #친구 #봄 #가을 #여름 #일출 #일몰 #밤 #올레 #오름 #해변 ...,0.0,7.0,따스한 햇살이 내리쬐는 세화에 위치한 오 세화 게스트하우스에는 두 동의 독채 펜션과...,oh세화


In [12]:
def adj_rows(main_idx, sub_idx):
    checker.at[main_idx, 'name'] = checker.at[sub_idx, 'name']
    checker.at[main_idx, 'tags'] = checker.at[sub_idx, 'tags']
    checker.at[main_idx, 'like'] = checker.at[sub_idx, 'like']
    checker.at[main_idx, 'heart'] = checker.at[sub_idx, 'heart']
    checker.at[main_idx, 'name_PRE'] = checker.at[sub_idx, 'name_PRE']

In [13]:
adj_rows(12, 998) # oh! 세화!
adj_rows(6, 1016) # for安
adj_rows(598, 1014) #오조리비앤비ojoribnb, 오조리비앤비
adj_rows(533, 1012) # 애월해안누리펜션구하얀둥지펜션, 애월해안누리펜션
adj_rows(747,985) # 제주하이킹inn구덕게스트하우스, 제주하이킹inn
adj_rows(336, 987) # 밸류호텔서귀포valuehotelseogwipojs, 밸류호텔서귀포
adj_rows(666, 988) # 제주돌집2호점스코리아, 제주돌집2호점
#취다선리조트 : 둘 다 name 값은 맞는데 비짓제주로그데이터 쪽에 관광지로 분류되어 있음 -> 처리X
adj_rows(950, 995) #호텔더블유신제주점hotelw, 호텔더블유신제주점
adj_rows(750, 997) # 제주해안휴양펜션제주해안펜션, 제주해안휴양펜션
adj_rows(565, 999) # 여울목게스트하우스월정리, 여울목게스트하우스
adj_rows(648, 1000) #제이드림호텔jdreamhotel, 제이드림호텔
adj_rows(870, 1001) # 풍경호텔풍경관광호텔, 풍경호텔
adj_rows(948, 1002) #호스텔린든hostellyndon, 호스텔린든
#관음사야영장 : 둘 다 name 값은 맞는데 비짓제주로그데이터 쪽에 관광지로 분류되어 있음 -> 처리X
adj_rows(175, 1004) # 돌고래하우스캠핑장, 돌고래하우스
adj_rows(235, 1006) # 리니민박riny, 리니민박
adj_rows(583, 1013) # 오션그랜드호텔함덕, 오션그랜드호텔 (오션그랜드호텔제주도 있는데 크롤링 데이터 오션그랜드호텔 주소랑 함덕이 같아서 함덕으로 편입)

In [14]:
final_lod = checker.dropna(subset = ['AREA_NM_PRE', 'name_PRE']).reset_index(drop=True)
final_lod = final_lod.drop(['name_PRE'], axis=1)
# final_lod.to_csv('././bigcon_data/숙소데이터_1_PRE.csv', encoding='utf-8-sig', index=False)
lodgement = final_lod.copy()
# lodgement = pd.read_csv('././bigcon_data/숙소데이터_1_PRE.csv')
lodgement = lodgement.rename({'name' : 'craw_name'}, axis=1)

## 2. 트립어드바이저 데이터로 가격정보 추가
- 트립어드바이저 크롤링 데이터에서 가격정보 가져오기

In [15]:
trip_adv_org = pd.read_csv("data/TRIPADVISOR_숙소크롤링.csv")
trip_adv = trip_adv_org.copy()

In [16]:
trip_adv['name_PRE'] = [ name if len(name.split('.'))==1 else ''.join(name.split('.')[1:]) for name in trip_adv.name ]

In [17]:
trip_adv = trip_adv.dropna(subset=['price']).reset_index(drop=True) # price가 없는 경우에는 drop
trip_adv = trip_adv.drop(['name'], axis=1) # 필요 없어서 제거

In [18]:
trip_adv

,link,price,name_PRE
0,/Hotel_Review-g297885-d9597798-Reviews-Aimi_Je...,"₩70,353",제주비치호텔 함덕
1,/Hotel_Review-g297885-d21364207-Reviews-Grand_...,"₩404,800",그랜드 하얏트 제주
2,/Hotel_Review-g297892-d11045667-Reviews-Golden...,"₩80,000",골든데이지 서귀포오션 호텔
3,/Hotel_Review-g297885-d455471-Reviews-Maison_G...,"₩201,600",메종 글래드 제주
4,/Hotel_Review-g297892-d7688313-Reviews-WE_Hote...,"₩294,000",WE호텔 제주
...,...,...,...
400,/Hotel_Review-g297885-d17390096-Reviews-Bijali...,"₩142,000",제주 비자림미담펜션
401,/Hotel_Review-g297885-d19650485-Reviews-Haean_...,"₩94,801",해안풍경펜션
402,/Hotel_Review-g297885-d7930841-Reviews-Spabell...,"₩240,204",스파벨리스
403,/Hotel_Review-g297885-d19116348-Reviews-Joy_Hi...,"₩68,166",조이힐하우스


In [19]:
#현재 str으로 되어 있는 price를 int로 바꿔줌
trip_adv['price'] = [int(price.replace('₩','').replace(',','')) for price in trip_adv['price']]
trip_adv['name'] = trip_adv['name_PRE'][:]

In [20]:
def name_preprocessing_hotel(name):
    name = name.replace(' (', '(').replace(') ', ')').strip()
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[\(\)㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = re.sub('[\[.\]]','',name)
    name = name.replace("호텔", "")
    return name.strip()

In [21]:
trip_adv['name_PRE'] = trip_adv['name'].apply(name_preprocessing_hotel)

In [22]:
def ta_grouping(x):
    return pd.Series({
                      'name' : x['name'].unique(),
                      'link' : x['link'].unique()[0],
                      'price' : int(x['price'].mean())})

In [23]:
trip_adv = trip_adv.groupby(['name_PRE']).apply(ta_grouping).reset_index() # 여러개 있는 경우 평균 값으로 만들어버림
trip_adv

,name_PRE,name,link,price
0,52,[ 호텔52],/Hotel_Review-g297892-d12695152-Reviews-Hotel_...,75000
1,aroomihotelhyupjae,[Aroomi Hotel Hyupjae],/Hotel_Review-g297885-d23590390-Reviews-Aroomi...,81360
2,dk하우스,[ DK하우스],/Hotel_Review-g297892-d16494190-Reviews-DK_Hou...,64000
3,gudeokguesthouse,[ Gudeok Guesthouse],/Hotel_Review-g297892-d9787080-Reviews-Gudeok_...,42355
4,hotelbloom,[ Hotel Bloom],/Hotel_Review-g297892-d1163838-Reviews-Hotel_B...,165722
...,...,...,...,...
363,휴식서귀포,[ 호텔 휴식 서귀포],/Hotel_Review-g297892-d9457738-Reviews-Hotel_r...,62400
364,휴식시드니,[ 휴식 호텔 시드니],/Hotel_Review-g297885-d11952258-Reviews-Huesik...,50045
365,휴안스테이,[ 휴안스테이 호텔],/Hotel_Review-g297892-d12586704-Reviews-Hu_An_...,63044
366,흰수염고래리조트,[ 흰수염고래리조트],/Hotel_Review-g297885-d10038251-Reviews-Blue_W...,105000


In [24]:
lodgement['hotel_remove_name'] = lodgement['AREA_NM_PRE'].apply(name_preprocessing_hotel)
lodgement['AREA_NM_PRE'].value_counts()

나바론민박        2
1915지오하우스    1
제주나인리조트      1
절물길펜션        1
제니빌펜션        1
            ..
발리인제주        1
밧돌게스트하우스     1
방울풍뎅이하우스     1
밭담사이         1
힐링캠프펜션       1
Name: AREA_NM_PRE, Length: 918, dtype: int64

In [25]:
lodgement

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,craw_name,tags,like,heart,detail_text,hotel_remove_name
0,1915지오하우스,숙박,['1915 지오하우스'],['제주특별자치도 서귀포시 성산읍 성산등용로 14'],12129.0,38.0,43.0,33.0,44.0,40.0,47.0,1915 지오하우스,#휴식 #숙소 #민박 #고향민박 #게스트하우스 #지질트레일 #체험,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,1915지오하우스
1,521게스트하우스,숙박,['521 게스트 하우스'],['제주특별자치도 제주시 애월읍 중용길 52-1 (신엄리) 521 게스트 하우스'],12129.0,5.0,7.0,12.0,26.0,28.0,48.0,521 게스트 하우스,#안전인증숙소 #숙소 #게스트하우스 #조식포함 #안전인증민박,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,521게스트하우스
2,belazybb,숙박,['be Lazy B&B'],['제주특별자치도 서귀포시 남원읍 위미해안로129번길 17'],12129.0,13.0,11.0,18.0,23.0,20.0,23.0,be Lazy B&B,#휴식 #숙소 #일반숙박 #돌담길 #카페 #힐링쉼터 #펜션 #돌담길 #카페 #힐링쉼...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,belazybb
3,bk호텔제주,숙박,['BK호텔제주'],['제주특별자치도 서귀포시 칠십리로91번길 12'],12129.0,9.0,16.0,15.0,11.0,15.0,13.0,BK호텔제주,#호텔 #숙소 #비즈니스센터 #컨시어지서비스 #엘리베이터,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,bk제주
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],['제주특별자치도 서귀포시 이어도로 450'],12129.0,11.0,15.0,16.0,21.0,20.0,16.0,cafe말/개스트하우스 말,#휴식 #게스트하우스 #민박 #반려동물보호소 #조식 #카페 #반려동물보호소 #조식 #카페,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,cafe말개스트하우스말
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,휴마루리조트,숙박,['휴마루리조트'],['제주특별자치도 서귀포시 중산간동로8183번길 16'],12129.0,16.0,14.0,20.0,24.0,28.0,29.0,휴마루리조트,#휴식/치유 #숙소 #리조트 #휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전 ...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",휴마루리조트
915,흰고래게스트하우스,숙박,['흰고래게스트하우스'],['제주특별자치도 서귀포시 이어도로1066번길 26'],12129.0,17.0,12.0,18.0,32.0,22.0,25.0,흰고래게스트하우스,#게스트하우스 #숙소 #독채 #공공와이파이존 #가족 #돌담길,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,흰고래게스트하우스
916,흰수염고래리조트,숙박,['흰수염고래리조트'],['제주특별자치도 제주시 애월읍 일주서로 6818'],12129.0,10.0,28.0,20.0,46.0,64.0,59.0,흰수염고래리조트,#휴식 #숙소 #리조트 #주차장 #공공와이파이존 #편의시설 #휴양콘도 #아동놀이방,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,흰수염고래리조트
917,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],['제주특별자치도 서귀포시 예래해안로 542'],12129.0,106.0,124.0,135.0,96.0,107.0,101.0,히든클리프 호텔 앤 네이쳐,#휴식 #5성급호텔 #숙소 #호텔 #공공와이파이존 #수영장 #주차장 #관광호텔 #호...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,히든클리프앤네이쳐


In [26]:
tmp_lodgement = pd.merge(left=lodgement, right = trip_adv, left_on = 'hotel_remove_name', right_on = 'name_PRE', how = 'left')
tmp_lodgement=tmp_lodgement.rename({'name' : 'ta_name'}, axis=1)

In [27]:
tmp_lodgement

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,craw_name,tags,like,heart,detail_text,hotel_remove_name,name_PRE,ta_name,link,price
0,1915지오하우스,숙박,['1915 지오하우스'],['제주특별자치도 서귀포시 성산읍 성산등용로 14'],12129.0,38.0,43.0,33.0,44.0,40.0,...,1915 지오하우스,#휴식 #숙소 #민박 #고향민박 #게스트하우스 #지질트레일 #체험,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,1915지오하우스,NaN,NaN,NaN,NaN
1,521게스트하우스,숙박,['521 게스트 하우스'],['제주특별자치도 제주시 애월읍 중용길 52-1 (신엄리) 521 게스트 하우스'],12129.0,5.0,7.0,12.0,26.0,28.0,...,521 게스트 하우스,#안전인증숙소 #숙소 #게스트하우스 #조식포함 #안전인증민박,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,521게스트하우스,NaN,NaN,NaN,NaN
2,belazybb,숙박,['be Lazy B&B'],['제주특별자치도 서귀포시 남원읍 위미해안로129번길 17'],12129.0,13.0,11.0,18.0,23.0,20.0,...,be Lazy B&B,#휴식 #숙소 #일반숙박 #돌담길 #카페 #힐링쉼터 #펜션 #돌담길 #카페 #힐링쉼...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,belazybb,NaN,NaN,NaN,NaN
3,bk호텔제주,숙박,['BK호텔제주'],['제주특별자치도 서귀포시 칠십리로91번길 12'],12129.0,9.0,16.0,15.0,11.0,15.0,...,BK호텔제주,#호텔 #숙소 #비즈니스센터 #컨시어지서비스 #엘리베이터,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,bk제주,NaN,NaN,NaN,NaN
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],['제주특별자치도 서귀포시 이어도로 450'],12129.0,11.0,15.0,16.0,21.0,20.0,...,cafe말/개스트하우스 말,#휴식 #게스트하우스 #민박 #반려동물보호소 #조식 #카페 #반려동물보호소 #조식 #카페,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,cafe말개스트하우스말,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,휴마루리조트,숙박,['휴마루리조트'],['제주특별자치도 서귀포시 중산간동로8183번길 16'],12129.0,16.0,14.0,20.0,24.0,28.0,...,휴마루리조트,#휴식/치유 #숙소 #리조트 #휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전 ...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",휴마루리조트,휴마루리조트,[ 휴마루 리조트],/Hotel_Review-g297892-d8327073-Reviews-Hyumaru...,76635.0
915,흰고래게스트하우스,숙박,['흰고래게스트하우스'],['제주특별자치도 서귀포시 이어도로1066번길 26'],12129.0,17.0,12.0,18.0,32.0,22.0,...,흰고래게스트하우스,#게스트하우스 #숙소 #독채 #공공와이파이존 #가족 #돌담길,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,흰고래게스트하우스,NaN,NaN,NaN,NaN
916,흰수염고래리조트,숙박,['흰수염고래리조트'],['제주특별자치도 제주시 애월읍 일주서로 6818'],12129.0,10.0,28.0,20.0,46.0,64.0,...,흰수염고래리조트,#휴식 #숙소 #리조트 #주차장 #공공와이파이존 #편의시설 #휴양콘도 #아동놀이방,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,흰수염고래리조트,흰수염고래리조트,[ 흰수염고래리조트],/Hotel_Review-g297885-d10038251-Reviews-Blue_W...,105000.0
917,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],['제주특별자치도 서귀포시 예래해안로 542'],12129.0,106.0,124.0,135.0,96.0,107.0,...,히든클리프 호텔 앤 네이쳐,#휴식 #5성급호텔 #숙소 #호텔 #공공와이파이존 #수영장 #주차장 #관광호텔 #호...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,히든클리프앤네이쳐,NaN,NaN,NaN,NaN


In [28]:
tmp_lodgement[tmp_lodgement['AREA_NM_PRE']=='나바론민박'] # 같은게 두개 있어서 아래쪽으로 합해줌

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,craw_name,tags,like,heart,detail_text,hotel_remove_name,name_PRE,ta_name,link,price
91,나바론민박,숙박,['나바론 민박' '나바론민박'],['제주특별자치도 제주시 추자면 추자로 76-1'],24258.0,35.0,33.0,34.0,55.0,39.0,...,나바론 민박,#추자나바론 #바비큐장 #추자도 #숙소 #민박 #농어촌민박 #특산품판매장,0.0,0.0,숙소 창 밖으로 보이는 바다뷰 세탁서비스와 바비큐장 및 픽업서비스 이용 가능하다,나바론민박,NaN,NaN,NaN,NaN
92,나바론민박,숙박,['나바론 민박' '나바론민박'],['제주특별자치도 제주시 추자면 추자로 76-1'],24258.0,35.0,33.0,34.0,55.0,39.0,...,나바론민박,#민박 #숙소 #농어촌민박 #교통 #조식 #음식 #주차장 #공공와이파이존 #교통 #...,0.0,1.0,"아름다운 섬 추자도에 위치한 민박집이다. 제주 앞바다를 품에 안은 아름다움, 내집 ...",나바론민박,NaN,NaN,NaN,NaN


In [29]:
# 나바론민박 바꿔줌
tmp_lodgement = tmp_lodgement.drop([91], axis=0).reset_index(drop=True)
tmp_lodgement = tmp_lodgement.drop(['ta_name', 'link'], axis=1)

In [30]:
del_list = []
for idx, name in enumerate(trip_adv['name_PRE']):
    if name in list(tmp_lodgement['hotel_remove_name']):
        del_list.append(idx)
trip_adv = trip_adv.drop(del_list, axis=0)

In [31]:
def adjust_price(vj_name, ta_name):
    vj_idx = tmp_lodgement[tmp_lodgement['AREA_NM_PRE'] == vj_name].index[0]
    ta_idx = trip_adv[trip_adv['name_PRE'] == ta_name].index[0]
    tmp_lodgement.at[vj_idx, 'price'] = trip_adv.iloc[ta_idx]['price']

In [32]:
adjust_price('제주we호텔', 'we제주')
adjust_price('라마다앙코르서귀포호텔', '라마다앙코르제주서귀포')
# adjust_price('해비치호텔앤드리조트', '해비치리조트제주')

adjust_price('유탑유블레스호텔', '유탑유블레스제주')
adjust_price('신라호텔제주', '제주신라')
adjust_price('라마다앙코르서귀포이스트호텔', '라마다앙코르이스트')
adjust_price('오션그랜드호텔함덕', '오션그랜드제주')
adjust_price('제주오리엔탈호텔', '제주오리엔탈앤카지노')
# adjust_price('제주펄관광호텔', '펄제주')
adjust_price('호텔스카브로', '스카보로') # 주소 같음
adjust_price('썬라이즈호텔', '썬라이즈-섭지코지점')
adjust_price('와이리조트제주', 'y리조트제주')
adjust_price('루체빌', '루체빌리조트')
adjust_price('bk호텔제주', '비케이제주') # 주소 같음
adjust_price('신라호텔제주', '제주신라')
adjust_price('서머셋제주신화월드호텔앤리조트', '서머셋제주신화월드')
# adjust_price('캠퍼트리호텔앤리조트', '캠퍼트리리조트')
adjust_price('아름다운리조트', '제주아름다운리조트')

adjust_price('파밀리아호텔', '파밀리아호텔제주')
adjust_price('미르빌펜션', '미르빌펜션리조트')
adjust_price('제주인호텔', '제주인관광') # 주소 같음
# adjust_price('중문스테이', '중문스테이-제주')
adjust_price('돈내코힐리조트', '돈내코힐스파리조트')
# adjust_price('풍경호텔풍경관광호텔', '풍경')
adjust_price('엠버리조트', '엠버리조트제주')
adjust_price('아름다운리조트', '제주아름다운리조트')

adjust_price('펜션형민박꿈꾸는숲', '제주꿈꾸는숲펜션')
adjust_price('제주하이킹inn구덕게스트하우스', 'gudeokguesthouse') # 주소 같음
adjust_price('블룸호텔', 'hotelbloom')
adjust_price('호텔더블유탑동점', 'w탑동점')
adjust_price('제주하이킹inn구덕게스트하우스', '구덕게스트하우스') # 주소 같음
adjust_price('그라벨호텔제주', '그라벨')
adjust_price('꿈꾸는노마드', '꿈꾸는노마드펜션')
adjust_price('돈내코힐리조트', '돈내코힐스파리조트')
adjust_price('동원리조트제주', '동원리조트')
adjust_price('디오션힐', '디오션힐펜션')
adjust_price('라마다앙코르서귀포호텔', '라마다앙코르제주서귀포')
adjust_price('라온호텔앤리조트', '라온리조트')
adjust_price('라이온힐펜션', '라이온힐휴양펜션')

adjust_price('마니주팬션', '마니주펜션')
adjust_price('마레보리조트', '마레보비치')
adjust_price('마빈', '마빈과르마빈펜션')
adjust_price('제주메이더호텔카라반빌리지', '메이더카라반')
adjust_price('제주메이플호텔', '메이플')
adjust_price('미르빌펜션', '미르빌펜션리조트')
adjust_price('바다드림', '바다드림펜션')
adjust_price('바닷가하얀집', '바닷가하얀집펜션')
adjust_price('바사팬스파', '바사팬펜션')
adjust_price('제주베스트힐펜션글램핑', '베스트힐펜션글램핑')
adjust_price('비스타리조트', '비스타뷰리조트')
adjust_price('미소가펜션', '서귀포미소가펜션')
adjust_price('서머셋제주신화월드호텔앤리조트', '서머셋제주신화월드')
adjust_price('소랑호젠', '소랑호젠펜션')
adjust_price('수앤수가족호텔', '수앤수리조트')
adjust_price('로베로호텔', '스타즈제주로베로')
adjust_price('제주시드니호텔', '시드니')
adjust_price('아뜨네통나무펜션', '아뜨네제주통나무펜션')
adjust_price('예이츠산장', '예이츠빌')
adjust_price('오션그랜드호텔함덕', '오션그랜드제주')
adjust_price('오션스퀘어', '오션스퀘어리조트')
adjust_price('유탑유블레스호텔', '유탑유블레스제주')
adjust_price('저스트슬립제주', '저스트슬립펜션')
adjust_price('또올레펜션게스트하우스', '제주또올레')
adjust_price('제주오리엔탈호텔', '제주오리엔탈앤카지노')
adjust_price('팜힐', '제주팜힐')
# adjust_price('호텔캘리포니아제주', '캘리포니아')
# adjust_price('캠퍼트리호텔앤리조트', '캠퍼트리리조트')
# adjust_price('코델리아리조트', '코델리아s')
# adjust_price('킴스캐빈', '킴스캐빈게스트하우스')
# adjust_price('팜밸리리조트', '팜밸리풀빌라리조트')
# adjust_price('패밀리아팬션', '패밀리아펜션')
# adjust_price('제이드림호텔jdreamhotel', '풀로라제이드림호텔')
# adjust_price('휘닉스제주섭지코지', '휘닉스제주')
adjust_price('해비치호텔앤드리조트', '올레리조트스파')

In [33]:
tmp_lodgement = tmp_lodgement.drop(['name_PRE', 'hotel_remove_name'], axis=1)
# tmp_lodgement.to_csv('././bigcon_data/숙소데이터_2_PRE.csv', encoding='utf-8-sig', index=False)

In [34]:
tmp_lodgement

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,craw_name,tags,like,heart,detail_text,price
0,1915지오하우스,숙박,['1915 지오하우스'],['제주특별자치도 서귀포시 성산읍 성산등용로 14'],12129.0,38.0,43.0,33.0,44.0,40.0,47.0,1915 지오하우스,#휴식 #숙소 #민박 #고향민박 #게스트하우스 #지질트레일 #체험,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,NaN
1,521게스트하우스,숙박,['521 게스트 하우스'],['제주특별자치도 제주시 애월읍 중용길 52-1 (신엄리) 521 게스트 하우스'],12129.0,5.0,7.0,12.0,26.0,28.0,48.0,521 게스트 하우스,#안전인증숙소 #숙소 #게스트하우스 #조식포함 #안전인증민박,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,NaN
2,belazybb,숙박,['be Lazy B&B'],['제주특별자치도 서귀포시 남원읍 위미해안로129번길 17'],12129.0,13.0,11.0,18.0,23.0,20.0,23.0,be Lazy B&B,#휴식 #숙소 #일반숙박 #돌담길 #카페 #힐링쉼터 #펜션 #돌담길 #카페 #힐링쉼...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,NaN
3,bk호텔제주,숙박,['BK호텔제주'],['제주특별자치도 서귀포시 칠십리로91번길 12'],12129.0,9.0,16.0,15.0,11.0,15.0,13.0,BK호텔제주,#호텔 #숙소 #비즈니스센터 #컨시어지서비스 #엘리베이터,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,118800.0
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],['제주특별자치도 서귀포시 이어도로 450'],12129.0,11.0,15.0,16.0,21.0,20.0,16.0,cafe말/개스트하우스 말,#휴식 #게스트하우스 #민박 #반려동물보호소 #조식 #카페 #반려동물보호소 #조식 #카페,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,휴마루리조트,숙박,['휴마루리조트'],['제주특별자치도 서귀포시 중산간동로8183번길 16'],12129.0,16.0,14.0,20.0,24.0,28.0,29.0,휴마루리조트,#휴식/치유 #숙소 #리조트 #휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전 ...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",76635.0
914,흰고래게스트하우스,숙박,['흰고래게스트하우스'],['제주특별자치도 서귀포시 이어도로1066번길 26'],12129.0,17.0,12.0,18.0,32.0,22.0,25.0,흰고래게스트하우스,#게스트하우스 #숙소 #독채 #공공와이파이존 #가족 #돌담길,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,NaN
915,흰수염고래리조트,숙박,['흰수염고래리조트'],['제주특별자치도 제주시 애월읍 일주서로 6818'],12129.0,10.0,28.0,20.0,46.0,64.0,59.0,흰수염고래리조트,#휴식 #숙소 #리조트 #주차장 #공공와이파이존 #편의시설 #휴양콘도 #아동놀이방,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,105000.0
916,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],['제주특별자치도 서귀포시 예래해안로 542'],12129.0,106.0,124.0,135.0,96.0,107.0,101.0,히든클리프 호텔 앤 네이쳐,#휴식 #5성급호텔 #숙소 #호텔 #공공와이파이존 #수영장 #주차장 #관광호텔 #호...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,NaN


## 3.숙박시설 현황 데이터에서 객실 수 정보 추가
- 숙박시설 현황 세부내역 데이터에서 숙소 별 객실 수 데이터 추가

In [35]:
def number_room_data(sheet_name : int, type : str):
    data = pd.read_excel('././bigcon_data/5. 숙박시설 현황 세부내역(22.7.31.기준)_도수합.xlsx', sheet_name=sheet_name)
    data.columns = data.iloc[0].tolist()
    data = data.drop([0], axis=0).reset_index(drop=True)
    data['type'] = type
    return data

In [36]:
def number_room_data_2(sheet_name : int):
    data = pd.read_excel('././bigcon_data/5-1. 숙박시설(농어촌민박) 현황 세부내역(22.7.31.기준)_도수합.xlsx', sheet_name=sheet_name)
    data.columns = data.iloc[0].tolist()
    data = data.drop([0], axis=0).reset_index(drop=True)
    data['type'] = '농어촌민박'
    return data

In [37]:
data_1 = number_room_data(0, '관광호텔업')
data_2 = number_room_data(1, '한국전통호텔업')
data_3 = number_room_data(2, '가족호텔업')
data_4 = number_room_data(3, '호스텔업')
data_5 = number_room_data(4, '소형호텔업')
data_6 = number_room_data(5, '휴양콘도미니엄업')
data_7 = number_room_data(6, '휴양펜션업')
data_8 = number_room_data(7, '일반숙박업')
data_9 = number_room_data(8, '생활숙박업')
data_10 = number_room_data(10, '유스호스텔')
data = pd.concat([data_1, data_2, data_3, data_4, data_5, data_6, data_7, data_10], axis=0).reset_index(drop=True)

In [38]:
data_11 = number_room_data_2(1)
data_12 = number_room_data_2(2)
# data_8, data_9의 경우 가지고 있는 열이 달라서 따로 빼서 정리해줌
data_2 = pd.concat([data_8, data_9])
data_2.columns = data_2.iloc[0].tolist()
data_2 = data_2.drop([0], axis=0).reset_index(drop=True)
data_2 = data_2.rename({'일반숙박업' : 'type'}, axis=1)
# 농어촌도 가지고 있는 열 달라서 따로 빼줌
data_3 = pd.concat([data_11, data_12])
data_3 = data_3.drop(['연번', '행정시별', '비고'], axis=1)
data = data.drop(['대표자명', '행정시별', '연번', '연락처', '순번', '비고'], axis=1)
data_2 = data_2.drop(['연번', '행정시별', '비고'], axis=1)
# 합침
data = pd.concat([data, data_2, data_3], axis=0)
data = data.reset_index(drop=True)

In [39]:
data = data.dropna().reset_index(drop=True)

In [40]:
data['상호명_PRE']= [' '.join(list(map(name_preprocessing, x.split('(구')))) if '(구' in str(x) else name_preprocessing(str(x)) for x in data['상호명']]
data['상호명_PRE']

0                            호텔하니크라운
1       글래드호텔앤리조트메종글래드제주 오라관광메종글래드제주
2                              제주썬호텔
3                             제주로얄호텔
4                            제주팔레스호텔
                    ...             
6516                            스테이결
6517                           와르르맨션
6518                            아란치아
6519                             유니네
6520                       lunagemm2
Name: 상호명_PRE, Length: 6521, dtype: object

In [41]:
# 주소로 mapping하기 위한 전처리
import ast
tmp_lodgement['ADDR'] = tmp_lodgement['ADDR'].apply(ast.literal_eval) # 리스트가 아니라 '['여서 바꿔줌
print(tmp_lodgement['ADDR'].apply(len).unique()) # 다 len 1개라서 그냥 string으로 바꿔줌
tmp_lodgement['ADDR'] = tmp_lodgement['ADDR'].apply(lambda x : x[0])

[1]


In [42]:
data['소재지'] = [''.join(place.split('(')[:-1]).strip() if '(' in place else place for place in data['소재지']]
data['소재지'] = [addr.replace(' ', '').strip() for addr in data['소재지']]
tmp_lodgement['ADDR'] = [addr.replace('제주특별자치도', '').replace('제주도', '') .strip() for addr in tmp_lodgement['ADDR']]
tmp_lodgement['ADDR'] = [''.join(addr.split('(')[:-1]).strip() if '(' in addr else addr for addr in tmp_lodgement['ADDR']]
tmp_lodgement['ADDR'] = [addr.replace(' ', '').strip() for addr in tmp_lodgement['ADDR']]

In [43]:
tmp_lodgement = pd.merge(left = tmp_lodgement, right = data, left_on = 'ADDR', right_on = '소재지', how = 'left')
tmp_lodgement

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,tags,like,heart,detail_text,price,상호명,소재지,객실수,type,상호명_PRE
0,1915지오하우스,숙박,['1915 지오하우스'],서귀포시성산읍성산등용로14,12129.0,38.0,43.0,33.0,44.0,40.0,...,#휴식 #숙소 #민박 #고향민박 #게스트하우스 #지질트레일 #체험,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,NaN,1915지오,서귀포시성산읍성산등용로14,3,농어촌민박,1915지오
1,521게스트하우스,숙박,['521 게스트 하우스'],제주시애월읍중용길52-1,12129.0,5.0,7.0,12.0,26.0,28.0,...,#안전인증숙소 #숙소 #게스트하우스 #조식포함 #안전인증민박,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,NaN,가지스테이,제주시애월읍중용길52-1,4,농어촌민박,가지스테이
2,belazybb,숙박,['be Lazy B&B'],서귀포시남원읍위미해안로129번길17,12129.0,13.0,11.0,18.0,23.0,20.0,...,#휴식 #숙소 #일반숙박 #돌담길 #카페 #힐링쉼터 #펜션 #돌담길 #카페 #힐링쉼...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,NaN,be Lazy,서귀포시남원읍위미해안로129번길17,2,농어촌민박,belazy
3,bk호텔제주,숙박,['BK호텔제주'],서귀포시칠십리로91번길12,12129.0,9.0,16.0,15.0,11.0,15.0,...,#호텔 #숙소 #비즈니스센터 #컨시어지서비스 #엘리베이터,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,118800.0,비케이호텔인제주,서귀포시칠십리로91번길12,55,관광호텔업,비케이호텔인제주
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],서귀포시이어도로450,12129.0,11.0,15.0,16.0,21.0,20.0,...,#휴식 #게스트하우스 #민박 #반려동물보호소 #조식 #카페 #반려동물보호소 #조식 #카페,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,휴마루리조트,숙박,['휴마루리조트'],서귀포시중산간동로8183번길16,12129.0,16.0,14.0,20.0,24.0,28.0,...,#휴식/치유 #숙소 #리조트 #휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전 ...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",76635.0,벙커호텔앤리조트,서귀포시중산간동로8183번길16,20,호스텔업,벙커호텔앤리조트
948,흰고래게스트하우스,숙박,['흰고래게스트하우스'],서귀포시이어도로1066번길26,12129.0,17.0,12.0,18.0,32.0,22.0,...,#게스트하우스 #숙소 #독채 #공공와이파이존 #가족 #돌담길,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,NaN,흰고래,서귀포시이어도로1066번길26,2,농어촌민박,흰고래
949,흰수염고래리조트,숙박,['흰수염고래리조트'],제주시애월읍일주서로6818,12129.0,10.0,28.0,20.0,46.0,64.0,...,#휴식 #숙소 #리조트 #주차장 #공공와이파이존 #편의시설 #휴양콘도 #아동놀이방,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,105000.0,흰수염고래리조트,제주시애월읍일주서로6818,24,호스텔업,흰수염고래리조트
950,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],서귀포시예래해안로542,12129.0,106.0,124.0,135.0,96.0,107.0,...,#휴식 #5성급호텔 #숙소 #호텔 #공공와이파이존 #수영장 #주차장 #관광호텔 #호...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,NaN,히든클리프호텔&네이쳐,서귀포시예래해안로542,250,관광호텔업,히든클리프호텔네이쳐


## 4. 마이리얼트립 데이터에서 가격 정보 추가

In [44]:
myrealtrip = pd.read_csv('data/MYREALTRIP_숙소정보크롤링.csv')

In [45]:
myrealtrip['address'] = [addr.replace(' ','') for addr in myrealtrip['address']]
lodgement = pd.merge(tmp_lodgement, myrealtrip, left_on = 'ADDR', right_on = 'address', how = 'left')

In [46]:
lodgement

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,소재지,객실수,type,상호명_PRE,link,name,address,price_y,star,star_ppl
0,1915지오하우스,숙박,['1915 지오하우스'],서귀포시성산읍성산등용로14,12129.0,38.0,43.0,33.0,44.0,40.0,...,서귀포시성산읍성산등용로14,3,농어촌민박,1915지오,NaN,NaN,NaN,NaN,NaN,NaN
1,521게스트하우스,숙박,['521 게스트 하우스'],제주시애월읍중용길52-1,12129.0,5.0,7.0,12.0,26.0,28.0,...,제주시애월읍중용길52-1,4,농어촌민박,가지스테이,NaN,NaN,NaN,NaN,NaN,NaN
2,belazybb,숙박,['be Lazy B&B'],서귀포시남원읍위미해안로129번길17,12129.0,13.0,11.0,18.0,23.0,20.0,...,서귀포시남원읍위미해안로129번길17,2,농어촌민박,belazy,NaN,NaN,NaN,NaN,NaN,NaN
3,bk호텔제주,숙박,['BK호텔제주'],서귀포시칠십리로91번길12,12129.0,9.0,16.0,15.0,11.0,15.0,...,서귀포시칠십리로91번길12,55,관광호텔업,비케이호텔인제주,https://www.myrealtrip.com/accommodations/stay...,서귀포 BK 호텔,서귀포시칠십리로91번길12,148500.0,4,NaN
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],서귀포시이어도로450,12129.0,11.0,15.0,16.0,21.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,휴마루리조트,숙박,['휴마루리조트'],서귀포시중산간동로8183번길16,12129.0,16.0,14.0,20.0,24.0,28.0,...,서귀포시중산간동로8183번길16,20,호스텔업,벙커호텔앤리조트,https://www.myrealtrip.com/accommodations/stay...,제주 벙커호텔앤리조트,서귀포시중산간동로8183번길16,155340.0,후,NaN
957,흰고래게스트하우스,숙박,['흰고래게스트하우스'],서귀포시이어도로1066번길26,12129.0,17.0,12.0,18.0,32.0,22.0,...,서귀포시이어도로1066번길26,2,농어촌민박,흰고래,NaN,NaN,NaN,NaN,NaN,NaN
958,흰수염고래리조트,숙박,['흰수염고래리조트'],제주시애월읍일주서로6818,12129.0,10.0,28.0,20.0,46.0,64.0,...,제주시애월읍일주서로6818,24,호스텔업,흰수염고래리조트,https://www.myrealtrip.com/accommodations/stay...,흰수염고래 리조트,제주시애월읍일주서로6818,210000.0,4,NaN
959,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],서귀포시예래해안로542,12129.0,106.0,124.0,135.0,96.0,107.0,...,서귀포시예래해안로542,250,관광호텔업,히든클리프호텔네이쳐,https://www.myrealtrip.com/accommodations/stay...,히든 클리프 호텔 앤 네이쳐,서귀포시예래해안로542,715920.0,4,NaN


In [47]:
lodgement['price'] = np.NaN

for idx in range(len(lodgement)):
    price_ta = lodgement.iloc[idx]['price_x']
    price_mt = lodgement.iloc[idx]['price_y']
    if (np.isnan(price_ta)==False) & (np.isnan(price_mt)==False):
        lodgement.at[idx, 'price'] = (price_ta + price_mt) // 2
    elif (np.isnan(price_ta)) & (np.isnan(price_mt)==False):
        lodgement.at[idx, 'price'] = price_mt
    elif (np.isnan(price_ta)==False) & (np.isnan(price_mt)):
        lodgement.at[idx, 'price'] = price_ta
    else:
        pass

In [48]:
tmp_lodgement = lodgement.drop(['name', 'address', 'craw_name', '소재지', 'price_x', 'price_y', 'star_ppl', 'star', '상호명_PRE', 'link'], axis=1)

In [49]:
tmp_lodgement

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,tags,like,heart,detail_text,상호명,객실수,type,price
0,1915지오하우스,숙박,['1915 지오하우스'],서귀포시성산읍성산등용로14,12129.0,38.0,43.0,33.0,44.0,40.0,47.0,#휴식 #숙소 #민박 #고향민박 #게스트하우스 #지질트레일 #체험,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,1915지오,3,농어촌민박,NaN
1,521게스트하우스,숙박,['521 게스트 하우스'],제주시애월읍중용길52-1,12129.0,5.0,7.0,12.0,26.0,28.0,48.0,#안전인증숙소 #숙소 #게스트하우스 #조식포함 #안전인증민박,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,가지스테이,4,농어촌민박,NaN
2,belazybb,숙박,['be Lazy B&B'],서귀포시남원읍위미해안로129번길17,12129.0,13.0,11.0,18.0,23.0,20.0,23.0,#휴식 #숙소 #일반숙박 #돌담길 #카페 #힐링쉼터 #펜션 #돌담길 #카페 #힐링쉼...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,be Lazy,2,농어촌민박,NaN
3,bk호텔제주,숙박,['BK호텔제주'],서귀포시칠십리로91번길12,12129.0,9.0,16.0,15.0,11.0,15.0,13.0,#호텔 #숙소 #비즈니스센터 #컨시어지서비스 #엘리베이터,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,비케이호텔인제주,55,관광호텔업,133650.0
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],서귀포시이어도로450,12129.0,11.0,15.0,16.0,21.0,20.0,16.0,#휴식 #게스트하우스 #민박 #반려동물보호소 #조식 #카페 #반려동물보호소 #조식 #카페,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,휴마루리조트,숙박,['휴마루리조트'],서귀포시중산간동로8183번길16,12129.0,16.0,14.0,20.0,24.0,28.0,29.0,#휴식/치유 #숙소 #리조트 #휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전 ...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",벙커호텔앤리조트,20,호스텔업,115987.0
957,흰고래게스트하우스,숙박,['흰고래게스트하우스'],서귀포시이어도로1066번길26,12129.0,17.0,12.0,18.0,32.0,22.0,25.0,#게스트하우스 #숙소 #독채 #공공와이파이존 #가족 #돌담길,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,흰고래,2,농어촌민박,NaN
958,흰수염고래리조트,숙박,['흰수염고래리조트'],제주시애월읍일주서로6818,12129.0,10.0,28.0,20.0,46.0,64.0,59.0,#휴식 #숙소 #리조트 #주차장 #공공와이파이존 #편의시설 #휴양콘도 #아동놀이방,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,흰수염고래리조트,24,호스텔업,157500.0
959,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],서귀포시예래해안로542,12129.0,106.0,124.0,135.0,96.0,107.0,101.0,#휴식 #5성급호텔 #숙소 #호텔 #공공와이파이존 #수영장 #주차장 #관광호텔 #호...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,히든클리프호텔&네이쳐,250,관광호텔업,715920.0


In [50]:
# tmp_lodgement.to_csv('././bigcon_data/숙소데이터_3_PRE.csv', encoding='utf-8-sig', index=False)

## 5. 주소 변환, 좌표 추출
TMAP API 활용하여 지번주소 도로명주소 변환 및 좌표 추출

In [51]:
data = tmp_lodgement.copy()

In [52]:
tmap_api = "l7xx58498922010048bf89bf09cef7a0a81a"
logging.basicConfig(format='[%(levelname)s] %(asctime)s - %(message)s', level=logging.INFO, datefmt="%H:%M:%S")

In [53]:
version = 1
searchTypCd = 'NtoO'
reqMulti = 'S'
resCoordType = 'WGS84GEO'
appKey = tmap_api
callback = "result"

#
reqAdd = "서귀포시 성산읍 성산등용로 14"
params = {
    'version': version,
    'SearchTypCd': searchTypCd,
    'reqMulti': reqMulti,
    'appkey': tmap_api,
    'reqAdd': reqAdd
}

headers = {
    "appKey": appKey,
    "Accept": "application/json",
    "Content-Type": "application/json; charset=UTF-8"
}

In [54]:
resource_url = f'https://apis.openapi.sk.com/tmap/geo/convertAddress?version={version}&searchTypCd={searchTypCd}& reqAdd={reqAdd}&reqMulti={reqMulti}&resCoordType={resCoordType}&&callback = {callback}&appKey={appKey}'
req = requests.get(resource_url, params=params, headers=headers)
j = req.json()['ConvertAdd']
address = ' '.join(
    [j['middleDistName'], j['legalLowerDistName'], j['legalDetailName'], j['primary'] + '-' + j['secondary']])
lon, lat = j['oldLon'], j['oldLat']

In [55]:
data['lon'], data['lat'], data['지번주소'] = 0, 0, ''

In [56]:
def tmap_road_name(params, headers):
    resource_url = f'https://apis.openapi.sk.com/tmap/geo/convertAddress?version={version}&searchTypCd={searchTypCd}& reqAdd={reqAdd}&reqMulti={reqMulti}&resCoordType={resCoordType}&&callback = {callback}&appKey={appKey}'

    req = requests.get(resource_url, params=params, headers=headers)
    j = req.json()['ConvertAdd']
    address = ' '.join(
        [j['middleDistName'], j['legalLowerDistName'], j['legalDetailName'], j['primary'] + '-' + j['secondary']])
    lon, lat = j['oldLon'], j['oldLat']

    return address, lon, lat

In [57]:
for i in tqdm(range(len(data))):
    if data['지번주소'][i] != '':
        pass
    else:
        params['reqAdd'] = data['ADDR'][i]
        try :
            old_addr, lon, lat = tmap_road_name(params, headers)
        except :
            old_addr, lon, lat  = 'error', 'error', 'error'
        data.at[i, '지번주소'] = old_addr
        data.at[i, 'lon'] = lon
        data.at[i, 'lat'] = lat

100%|██████████| 961/961 [04:20<00:00,  3.69it/s]


In [58]:
data[data['lon']=='error'].index

Int64Index([62, 130, 137, 282, 611], dtype='int64')

In [59]:
# 에러나서 직접 검색해서 바꿔줌
data.at[62, '지번주소'] = '제주특별자치도 서귀포시 대정읍 신도리 2413'
data.at[130, '지번주소'] = '제주특별자치도 서귀포시 강정동 349-1'
data.at[130, 'ADDR'] = '제주특별자치도 서귀포시 월드컵로 84 씨월드펜션'
data.at[137, '지번주소'] ='제주특별자치도 제주시 한림읍 금능리 1593'
data.at[282, '지번주소'] = '제주특별자치도 제주시 애월읍 고내리 91'
data.at[611, '지번주소'] = '제주특별자치도 제주시 구좌읍 한동리 1358-4'

# 에러나서 손으로 채워줌 (구글맵스)
data.at[62, 'lon'], data.at[62, 'lat'] = '126.8306515', '33.5425546'
data.at[130, 'lon'], data.at[130, 'lat'] = '126.5049592', '33.2421749'
data.at[137, 'lon'], data.at[137, 'lat'] = '126.223733', '33.3883291'
data.at[282, 'lon'], data.at[282, 'lat'] = '126.3505064', '33.4713337'
data.at[611, 'lon'], data.at[611, 'lat'] = '126.8306515', '33.5425546'

In [60]:
data[data['lon']=='error'].index

Int64Index([], dtype='int64')

In [61]:
original_dont_touch = data.copy()

In [64]:
original_tmp_lod = tmp_lodgement.copy()

In [62]:
data

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,like,heart,detail_text,상호명,객실수,type,price,lon,lat,지번주소
0,1915지오하우스,숙박,['1915 지오하우스'],서귀포시성산읍성산등용로14,12129.0,38.0,43.0,33.0,44.0,40.0,...,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,1915지오,3,농어촌민박,NaN,126.93315783,33.46519626,서귀포시 성산읍 성산리 235-2
1,521게스트하우스,숙박,['521 게스트 하우스'],제주시애월읍중용길52-1,12129.0,5.0,7.0,12.0,26.0,28.0,...,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,가지스테이,4,농어촌민박,NaN,126.37178981,33.46885354,제주시 애월읍 신엄리 1369-16
2,belazybb,숙박,['be Lazy B&B'],서귀포시남원읍위미해안로129번길17,12129.0,13.0,11.0,18.0,23.0,20.0,...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,be Lazy,2,농어촌민박,NaN,126.65182553,33.27046324,서귀포시 남원읍 위미리 4207-1
3,bk호텔제주,숙박,['BK호텔제주'],서귀포시칠십리로91번길12,12129.0,9.0,16.0,15.0,11.0,15.0,...,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,비케이호텔인제주,55,관광호텔업,133650.0,126.56433378,33.24099288,서귀포시 서귀동 777-1
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],서귀포시이어도로450,12129.0,11.0,15.0,16.0,21.0,20.0,...,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,NaN,NaN,NaN,NaN,126.46723107,33.24154684,서귀포시 월평동 72-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,휴마루리조트,숙박,['휴마루리조트'],서귀포시중산간동로8183번길16,12129.0,16.0,14.0,20.0,24.0,28.0,...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",벙커호텔앤리조트,20,호스텔업,115987.0,126.54269631,33.26124030,서귀포시 서홍동 1777-3
957,흰고래게스트하우스,숙박,['흰고래게스트하우스'],서귀포시이어도로1066번길26,12129.0,17.0,12.0,18.0,32.0,22.0,...,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,흰고래,2,농어촌민박,NaN,126.52369834,33.24393635,서귀포시 서호동 65-2
958,흰수염고래리조트,숙박,['흰수염고래리조트'],제주시애월읍일주서로6818,12129.0,10.0,28.0,20.0,46.0,64.0,...,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,흰수염고래리조트,24,호스텔업,157500.0,126.38026116,33.47493633,제주시 애월읍 구엄리 941-0
959,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],서귀포시예래해안로542,12129.0,106.0,124.0,135.0,96.0,107.0,...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,히든클리프호텔&네이쳐,250,관광호텔업,715920.0,126.40237529,33.25484990,서귀포시 상예동 625-0


In [63]:
data['지번주소'] = [addr[:-2] if addr[-1] == '0' else addr for addr in data['지번주소']] # 지번주소 끝에 -0라는 불필요한 내용 있어서 정리
# data['지번주소'] = [addr.replace('제주특별자치도','') for addr in data['지번주소']]
data['지번주소'] = [re.sub(' +', ' ', addr).strip() for addr in data['지번주소']]
data

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,like,heart,detail_text,상호명,객실수,type,price,lon,lat,지번주소
0,1915지오하우스,숙박,['1915 지오하우스'],서귀포시성산읍성산등용로14,12129.0,38.0,43.0,33.0,44.0,40.0,...,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,1915지오,3,농어촌민박,NaN,126.93315783,33.46519626,서귀포시 성산읍 성산리 235-2
1,521게스트하우스,숙박,['521 게스트 하우스'],제주시애월읍중용길52-1,12129.0,5.0,7.0,12.0,26.0,28.0,...,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,가지스테이,4,농어촌민박,NaN,126.37178981,33.46885354,제주시 애월읍 신엄리 1369-16
2,belazybb,숙박,['be Lazy B&B'],서귀포시남원읍위미해안로129번길17,12129.0,13.0,11.0,18.0,23.0,20.0,...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,be Lazy,2,농어촌민박,NaN,126.65182553,33.27046324,서귀포시 남원읍 위미리 4207-1
3,bk호텔제주,숙박,['BK호텔제주'],서귀포시칠십리로91번길12,12129.0,9.0,16.0,15.0,11.0,15.0,...,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,비케이호텔인제주,55,관광호텔업,133650.0,126.56433378,33.24099288,서귀포시 서귀동 777-1
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],서귀포시이어도로450,12129.0,11.0,15.0,16.0,21.0,20.0,...,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,NaN,NaN,NaN,NaN,126.46723107,33.24154684,서귀포시 월평동 72-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,휴마루리조트,숙박,['휴마루리조트'],서귀포시중산간동로8183번길16,12129.0,16.0,14.0,20.0,24.0,28.0,...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",벙커호텔앤리조트,20,호스텔업,115987.0,126.54269631,33.26124030,서귀포시 서홍동 1777-3
957,흰고래게스트하우스,숙박,['흰고래게스트하우스'],서귀포시이어도로1066번길26,12129.0,17.0,12.0,18.0,32.0,22.0,...,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,흰고래,2,농어촌민박,NaN,126.52369834,33.24393635,서귀포시 서호동 65-2
958,흰수염고래리조트,숙박,['흰수염고래리조트'],제주시애월읍일주서로6818,12129.0,10.0,28.0,20.0,46.0,64.0,...,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,흰수염고래리조트,24,호스텔업,157500.0,126.38026116,33.47493633,제주시 애월읍 구엄리 941
959,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],서귀포시예래해안로542,12129.0,106.0,124.0,135.0,96.0,107.0,...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,히든클리프호텔&네이쳐,250,관광호텔업,715920.0,126.40237529,33.25484990,서귀포시 상예동 625


## 6. 지번단위 온실가스량 데이터 병합

In [65]:
co2 = pd.read_csv('data/PRE_CO2_지번단위탄소배출량_meanmax.csv')
lod = data.copy()

In [66]:
tmp_lodgement['lon'], tmp_lodgement['lat'], tmp_lodgement['지번주소'] = lod['lon'], lod['lat'], lod['지번주소']
# tmp_lodgement.to_csv('././bigcon_data/숙소데이터_4_PRE.csv')

In [67]:
tmp_lodgement

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,like,heart,detail_text,상호명,객실수,type,price,lon,lat,지번주소
0,1915지오하우스,숙박,['1915 지오하우스'],서귀포시성산읍성산등용로14,12129.0,38.0,43.0,33.0,44.0,40.0,...,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,1915지오,3,농어촌민박,NaN,126.93315783,33.46519626,서귀포시 성산읍 성산리 235-2
1,521게스트하우스,숙박,['521 게스트 하우스'],제주시애월읍중용길52-1,12129.0,5.0,7.0,12.0,26.0,28.0,...,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,가지스테이,4,농어촌민박,NaN,126.37178981,33.46885354,제주시 애월읍 신엄리 1369-16
2,belazybb,숙박,['be Lazy B&B'],서귀포시남원읍위미해안로129번길17,12129.0,13.0,11.0,18.0,23.0,20.0,...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,be Lazy,2,농어촌민박,NaN,126.65182553,33.27046324,서귀포시 남원읍 위미리 4207-1
3,bk호텔제주,숙박,['BK호텔제주'],서귀포시칠십리로91번길12,12129.0,9.0,16.0,15.0,11.0,15.0,...,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,비케이호텔인제주,55,관광호텔업,133650.0,126.56433378,33.24099288,서귀포시 서귀동 777-1
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],서귀포시이어도로450,12129.0,11.0,15.0,16.0,21.0,20.0,...,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,NaN,NaN,NaN,NaN,126.46723107,33.24154684,서귀포시 월평동 72-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,휴마루리조트,숙박,['휴마루리조트'],서귀포시중산간동로8183번길16,12129.0,16.0,14.0,20.0,24.0,28.0,...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",벙커호텔앤리조트,20,호스텔업,115987.0,126.54269631,33.26124030,서귀포시 서홍동 1777-3
957,흰고래게스트하우스,숙박,['흰고래게스트하우스'],서귀포시이어도로1066번길26,12129.0,17.0,12.0,18.0,32.0,22.0,...,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,흰고래,2,농어촌민박,NaN,126.52369834,33.24393635,서귀포시 서호동 65-2
958,흰수염고래리조트,숙박,['흰수염고래리조트'],제주시애월읍일주서로6818,12129.0,10.0,28.0,20.0,46.0,64.0,...,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,흰수염고래리조트,24,호스텔업,157500.0,126.38026116,33.47493633,제주시 애월읍 구엄리 941
959,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],서귀포시예래해안로542,12129.0,106.0,124.0,135.0,96.0,107.0,...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,히든클리프호텔&네이쳐,250,관광호텔업,715920.0,126.40237529,33.25484990,서귀포시 상예동 625


In [68]:
lod = tmp_lodgement.copy()

In [69]:
lod = lod.dropna(subset = ['lon', 'lat', '지번주소'], how = 'all').reset_index(drop=True)

In [70]:
lod

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,like,heart,detail_text,상호명,객실수,type,price,lon,lat,지번주소
0,1915지오하우스,숙박,['1915 지오하우스'],서귀포시성산읍성산등용로14,12129.0,38.0,43.0,33.0,44.0,40.0,...,0.0,2.0,1915지오하우스는 제주관광공사에서 유네스코 세계지질공원의 핵심 명소들이 위치한 지...,1915지오,3,농어촌민박,NaN,126.93315783,33.46519626,서귀포시 성산읍 성산리 235-2
1,521게스트하우스,숙박,['521 게스트 하우스'],제주시애월읍중용길52-1,12129.0,5.0,7.0,12.0,26.0,28.0,...,0.0,1.0,2인실 하루 렌트에 6만원(조식포함) 어르신들이 운영하고 있어 조용하고 여성분들이나...,가지스테이,4,농어촌민박,NaN,126.37178981,33.46885354,제주시 애월읍 신엄리 1369-16
2,belazybb,숙박,['be Lazy B&B'],서귀포시남원읍위미해안로129번길17,12129.0,13.0,11.0,18.0,23.0,20.0,...,1.0,3.0,be lazy는 B&B 스타일의 숙소로 한개의 더블룸과 세개의 트윈룸으로 구성되어 ...,be Lazy,2,농어촌민박,NaN,126.65182553,33.27046324,서귀포시 남원읍 위미리 4207-1
3,bk호텔제주,숙박,['BK호텔제주'],서귀포시칠십리로91번길12,12129.0,9.0,16.0,15.0,11.0,15.0,...,1.0,3.0,새섬과 칠십리 해안도로가 내려다 보이는 환상적인 조망권과 더불어 서귀포시에 위치해 ...,비케이호텔인제주,55,관광호텔업,133650.0,126.56433378,33.24099288,서귀포시 서귀동 777-1
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],서귀포시이어도로450,12129.0,11.0,15.0,16.0,21.0,20.0,...,0.0,0.0,'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하우스다. 이곳 마당에는 대...,NaN,NaN,NaN,NaN,126.46723107,33.24154684,서귀포시 월평동 72-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,휴마루리조트,숙박,['휴마루리조트'],서귀포시중산간동로8183번길16,12129.0,16.0,14.0,20.0,24.0,28.0,...,1.0,1.0,"휴마루리조트는 문섬, 섶섬, 범섬이 보이는 바다와 한라산 조망이 가능한 리조트이다....",벙커호텔앤리조트,20,호스텔업,115987.0,126.54269631,33.26124030,서귀포시 서홍동 1777-3
957,흰고래게스트하우스,숙박,['흰고래게스트하우스'],서귀포시이어도로1066번길26,12129.0,17.0,12.0,18.0,32.0,22.0,...,0.0,0.0,제주월드컵경기장이 있는 서귀포 시내의 위치한 흰고래 게스트하우스는 2층 건물로 이루...,흰고래,2,농어촌민박,NaN,126.52369834,33.24393635,서귀포시 서호동 65-2
958,흰수염고래리조트,숙박,['흰수염고래리조트'],제주시애월읍일주서로6818,12129.0,10.0,28.0,20.0,46.0,64.0,...,1.0,7.0,애월읍 구엄리에 위치한 흰수염고래 리조트는 동물의 모양을 한 테마객실과 야외 수영장...,흰수염고래리조트,24,호스텔업,157500.0,126.38026116,33.47493633,제주시 애월읍 구엄리 941
959,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],서귀포시예래해안로542,12129.0,106.0,124.0,135.0,96.0,107.0,...,1.0,37.0,2016년 7월에 개관한 히든클리프 호텔 앤 네이쳐는 제주의 원시적 생명력을 고스란...,히든클리프호텔&네이쳐,250,관광호텔업,715920.0,126.40237529,33.25484990,서귀포시 상예동 625


In [72]:
lod['지번주소'] = ['제주특별자치도 '+ addr+'번지' for addr in lod['지번주소']]

In [74]:
lod = pd.merge(left = lod, right = co2, left_on = '지번주소', right_on = 'LTNO_ADDR', how = 'left')

In [131]:
# lod['지번주소'] = ['제주특별자치도 '+ addr+'번지' for addr in lod['지번주소']]
# tmp_lodgement = pd.merge(left = lod, right = co2, left_on = '지번주소', right_on = 'LTNO_ADDR', how = 'left')
# # tmp_lodgement.to_csv('././bigcon_data/숙소데이터_5_PRE.csv', encoding='utf-8-sig', index=False)

# NA IMPUTATION

### co2의 경우 타입 별 평균, 가격의 경우 읍면동 별 타입 평균

In [75]:
lod

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,...,type,price,lon,lat,지번주소,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,co2_mean,co2_max
0,1915지오하우스,숙박,['1915 지오하우스'],서귀포시성산읍성산등용로14,12129.0,38.0,43.0,33.0,44.0,40.0,...,농어촌민박,NaN,126.93315783,33.46519626,제주특별자치도 서귀포시 성산읍 성산리 235-2번지,NaN,NaN,NaN,NaN,NaN
1,521게스트하우스,숙박,['521 게스트 하우스'],제주시애월읍중용길52-1,12129.0,5.0,7.0,12.0,26.0,28.0,...,농어촌민박,NaN,126.37178981,33.46885354,제주특별자치도 제주시 애월읍 신엄리 1369-16번지,NaN,NaN,NaN,NaN,NaN
2,belazybb,숙박,['be Lazy B&B'],서귀포시남원읍위미해안로129번길17,12129.0,13.0,11.0,18.0,23.0,20.0,...,농어촌민박,NaN,126.65182553,33.27046324,제주특별자치도 서귀포시 남원읍 위미리 4207-1번지,NaN,NaN,NaN,NaN,NaN
3,bk호텔제주,숙박,['BK호텔제주'],서귀포시칠십리로91번길12,12129.0,9.0,16.0,15.0,11.0,15.0,...,관광호텔업,133650.0,126.56433378,33.24099288,제주특별자치도 서귀포시 서귀동 777-1번지,제주특별자치도 서귀포시 서귀동 777-1번지,0.0,0.0,0.445723,0.581321
4,cafe말개스트하우스말,숙박,['cafe말/개스트하우스 말'],서귀포시이어도로450,12129.0,11.0,15.0,16.0,21.0,20.0,...,NaN,NaN,126.46723107,33.24154684,제주특별자치도 서귀포시 월평동 72-2번지,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,휴마루리조트,숙박,['휴마루리조트'],서귀포시중산간동로8183번길16,12129.0,16.0,14.0,20.0,24.0,28.0,...,호스텔업,115987.0,126.54269631,33.26124030,제주특별자치도 서귀포시 서홍동 1777-3번지,제주특별자치도 서귀포시 서홍동 1777-3번지,0.0,0.0,0.063011,0.092380
960,흰고래게스트하우스,숙박,['흰고래게스트하우스'],서귀포시이어도로1066번길26,12129.0,17.0,12.0,18.0,32.0,22.0,...,농어촌민박,NaN,126.52369834,33.24393635,제주특별자치도 서귀포시 서호동 65-2번지,NaN,NaN,NaN,NaN,NaN
961,흰수염고래리조트,숙박,['흰수염고래리조트'],제주시애월읍일주서로6818,12129.0,10.0,28.0,20.0,46.0,64.0,...,호스텔업,157500.0,126.38026116,33.47493633,제주특별자치도 제주시 애월읍 구엄리 941번지,제주특별자치도 제주시 애월읍 구엄리 941번지,0.0,0.0,0.244160,0.363100
962,히든클리프호텔앤네이쳐,숙박,['히든클리프 호텔 앤 네이쳐'],서귀포시예래해안로542,12129.0,106.0,124.0,135.0,96.0,107.0,...,관광호텔업,715920.0,126.40237529,33.25484990,제주특별자치도 서귀포시 상예동 625번지,NaN,NaN,NaN,NaN,NaN


In [125]:
tmp_lodgement = lod.drop(['AREA_NM', 'ADDR', 'BASE_YEAR'], axis=1)

In [126]:
lod_no_type = tmp_lodgement.dropna(subset=['type'])
lod_no_type = lod_no_type.reset_index(drop=True)

In [127]:
lod_no_type

,AREA_NM_PRE,CL_NM,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,tags,like,...,type,price,lon,lat,지번주소,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,co2_mean,co2_max
0,1915지오하우스,숙박,38.0,43.0,33.0,44.0,40.0,47.0,#휴식 #숙소 #민박 #고향민박 #게스트하우스 #지질트레일 #체험,0.0,...,농어촌민박,NaN,126.93315783,33.46519626,제주특별자치도 서귀포시 성산읍 성산리 235-2번지,NaN,NaN,NaN,NaN,NaN
1,521게스트하우스,숙박,5.0,7.0,12.0,26.0,28.0,48.0,#안전인증숙소 #숙소 #게스트하우스 #조식포함 #안전인증민박,0.0,...,농어촌민박,NaN,126.37178981,33.46885354,제주특별자치도 제주시 애월읍 신엄리 1369-16번지,NaN,NaN,NaN,NaN,NaN
2,belazybb,숙박,13.0,11.0,18.0,23.0,20.0,23.0,#휴식 #숙소 #일반숙박 #돌담길 #카페 #힐링쉼터 #펜션 #돌담길 #카페 #힐링쉼...,1.0,...,농어촌민박,NaN,126.65182553,33.27046324,제주특별자치도 서귀포시 남원읍 위미리 4207-1번지,NaN,NaN,NaN,NaN,NaN
3,bk호텔제주,숙박,9.0,16.0,15.0,11.0,15.0,13.0,#호텔 #숙소 #비즈니스센터 #컨시어지서비스 #엘리베이터,1.0,...,관광호텔업,133650.0,126.56433378,33.24099288,제주특별자치도 서귀포시 서귀동 777-1번지,제주특별자치도 서귀포시 서귀동 777-1번지,0.0,0.0,0.445723,0.581321
4,cs빌리지,숙박,23.0,28.0,33.0,22.0,16.0,22.0,#펜션 #숙소 #휴양펜션 #해안도로 #단체여행객 #독채 #바비큐 #온돌방 #가족 #...,0.0,...,농어촌민박,210000.0,126.36020727,33.47843562,제주특별자치도 제주시 애월읍 신엄리 2758번지,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,휘슬락호텔,숙박,6.0,11.0,10.0,17.0,16.0,15.0,#호텔 #숙소 #오션뷰호텔 #공공와이파이존 #수영장 #주차장 #양식레스토랑 #조식포함,2.0,...,일반숙박업,285000.0,126.52752517,33.51735064,제주특별자치도 제주시 건입동 1443-2번지,제주특별자치도 제주시 건입동 1443-2번지,0.0,0.0,2.225807,3.073150
738,휴마루리조트,숙박,16.0,14.0,20.0,24.0,28.0,29.0,#휴식/치유 #숙소 #리조트 #휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전 ...,1.0,...,호스텔업,115987.0,126.54269631,33.26124030,제주특별자치도 서귀포시 서홍동 1777-3번지,제주특별자치도 서귀포시 서홍동 1777-3번지,0.0,0.0,0.063011,0.092380
739,흰고래게스트하우스,숙박,17.0,12.0,18.0,32.0,22.0,25.0,#게스트하우스 #숙소 #독채 #공공와이파이존 #가족 #돌담길,0.0,...,농어촌민박,NaN,126.52369834,33.24393635,제주특별자치도 서귀포시 서호동 65-2번지,NaN,NaN,NaN,NaN,NaN
740,흰수염고래리조트,숙박,10.0,28.0,20.0,46.0,64.0,59.0,#휴식 #숙소 #리조트 #주차장 #공공와이파이존 #편의시설 #휴양콘도 #아동놀이방,1.0,...,호스텔업,157500.0,126.38026116,33.47493633,제주특별자치도 제주시 애월읍 구엄리 941번지,제주특별자치도 제주시 애월읍 구엄리 941번지,0.0,0.0,0.244160,0.363100


In [128]:
tmp = lod_no_type.dropna(subset=['type', 'co2_max'], how = 'any') ; tmp

,AREA_NM_PRE,CL_NM,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,tags,like,...,type,price,lon,lat,지번주소,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,co2_mean,co2_max
3,bk호텔제주,숙박,9.0,16.0,15.0,11.0,15.0,13.0,#호텔 #숙소 #비즈니스센터 #컨시어지서비스 #엘리베이터,1.0,...,관광호텔업,133650.0,126.56433378,33.24099288,제주특별자치도 서귀포시 서귀동 777-1번지,제주특별자치도 서귀포시 서귀동 777-1번지,0.0000,0.000,0.445723,0.581321
6,jaychloe,숙박,14.0,18.0,17.0,33.0,38.0,47.0,#휴식 #숙소 #펜션 #휴양펜션 #스파 #바비큐 #공공와이파이존 #조식포함 #해변 ...,0.0,...,호스텔업,NaN,126.18883822,33.25717963,제주특별자치도 서귀포시 대정읍 무릉리 3869-4번지,제주특별자치도 서귀포시 대정읍 무릉리 3869-4번지,0.0000,0.000,0.162573,0.265732
7,jj하우스,숙박,34.0,55.0,44.0,40.0,39.0,45.0,#펜션 #숙소 #휴양펜션 #온돌방 #가족 #바비큐 #독채 #공공와이파이존 #계곡 #...,0.0,...,호스텔업,NaN,126.48564572,33.26043393,제주특별자치도 서귀포시 강정동 3786-1번지,제주특별자치도 서귀포시 강정동 3786-1번지,0.0000,0.000,0.063178,0.143558
8,jk라마다앙코르제주연동호텔,숙박,13.0,25.0,29.0,17.0,43.0,34.0,#호텔 #숙소 #편의시설 #교통 #비즈니스센터 #카페 #조식 #연회장,2.0,...,일반숙박업,271860.0,126.49055674,33.49213062,제주특별자치도 제주시 연동 270-2번지,제주특별자치도 제주시 연동 270-2번지,906225.4312,1500497.791,0.981347,1.524153
13,가산토방,숙박,14.0,19.0,15.0,24.0,32.0,28.0,#휴식 #휴양펜션 #펜션 #한옥 #전통호텔 #독채 #공공와이파이존 #온돌방 #가족 ...,0.0,...,호스텔업,NaN,126.57763713,33.28765461,제주특별자치도 서귀포시 토평동 3077번지,제주특별자치도 서귀포시 토평동 3077번지,0.0000,0.000,0.038196,0.087593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,화인호텔,숙박,49.0,53.0,56.0,30.0,26.0,38.0,#호텔 #숙소 #게스트하우스 #4성급호텔 #양식레스토랑 #비즈니스센터 #관광호텔,4.0,...,관광호텔업,181500.0,126.56511150,33.24046517,제주특별자치도 서귀포시 서귀동 782-3번지,제주특별자치도 서귀포시 서귀동 782-3번지,0.0000,0.000,0.531999,1.238175
736,휘닉스제주섭지코지,숙박,230.0,226.0,256.0,195.0,245.0,299.0,#리조트 #성산 #4성급호텔 #수영장 #휴양콘도 #주차장 #공공와이파이존 #어린이놀...,2.0,...,휴양콘도미니엄업,NaN,126.92732581,33.43003338,제주특별자치도 서귀포시 성산읍 고성리 127-2번지,제주특별자치도 서귀포시 성산읍 고성리 127-2번지,0.0000,0.000,7.338803,10.724319
737,휘슬락호텔,숙박,6.0,11.0,10.0,17.0,16.0,15.0,#호텔 #숙소 #오션뷰호텔 #공공와이파이존 #수영장 #주차장 #양식레스토랑 #조식포함,2.0,...,일반숙박업,285000.0,126.52752517,33.51735064,제주특별자치도 제주시 건입동 1443-2번지,제주특별자치도 제주시 건입동 1443-2번지,0.0000,0.000,2.225807,3.073150
738,휴마루리조트,숙박,16.0,14.0,20.0,24.0,28.0,29.0,#휴식/치유 #숙소 #리조트 #휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전 ...,1.0,...,호스텔업,115987.0,126.54269631,33.26124030,제주특별자치도 서귀포시 서홍동 1777-3번지,제주특별자치도 서귀포시 서홍동 1777-3번지,0.0000,0.000,0.063011,0.092380


In [129]:
lod_no_type['읍면동'] = [lod_no_type.split('시')[1].strip().split(' ')[0] for lod_no_type in lod_no_type['지번주소']]
address_price_mean = lod_no_type.groupby(['읍면동', 'type'])['price'].mean().to_dict()
type_co2_max_mean = lod_no_type.groupby(['type'])['co2_max'].mean().to_dict()

In [130]:
for idx in range(len(tmp)):
    if np.isnan(lod_no_type.iloc[idx]['price']):
        lod_no_type.at[idx, 'price'] = address_price_mean[(lod_no_type.iloc[idx]['읍면동'], lod_no_type.iloc[idx]['type'])]
    if np.isnan(lod_no_type.iloc[idx]['co2_max']):
        lod_no_type.at[idx, 'co2_max'] = type_co2_max_mean[lod_no_type.iloc[idx]['type']]

In [122]:
lod_no_type = lod_no_type.dropna(subset=['price', 'co2_max']).reset_index(drop=True)

In [131]:
# 객실 수로 나누기
lod_no_type['co2_max_객실'] = [ y/x for x, y in zip(lod_no_type['객실수'], lod_no_type['co2_max'])]

In [132]:
lod_no_type

,AREA_NM_PRE,CL_NM,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO,tags,like,...,lon,lat,지번주소,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,co2_mean,co2_max,읍면동,co2_max_객실
0,1915지오하우스,숙박,38.0,43.0,33.0,44.0,40.0,47.0,#휴식 #숙소 #민박 #고향민박 #게스트하우스 #지질트레일 #체험,0.0,...,126.93315783,33.46519626,제주특별자치도 서귀포시 성산읍 성산리 235-2번지,NaN,NaN,NaN,NaN,0.064933,성산읍,0.021644
1,521게스트하우스,숙박,5.0,7.0,12.0,26.0,28.0,48.0,#안전인증숙소 #숙소 #게스트하우스 #조식포함 #안전인증민박,0.0,...,126.37178981,33.46885354,제주특별자치도 제주시 애월읍 신엄리 1369-16번지,NaN,NaN,NaN,NaN,0.064933,애월읍,0.016233
2,belazybb,숙박,13.0,11.0,18.0,23.0,20.0,23.0,#휴식 #숙소 #일반숙박 #돌담길 #카페 #힐링쉼터 #펜션 #돌담길 #카페 #힐링쉼...,1.0,...,126.65182553,33.27046324,제주특별자치도 서귀포시 남원읍 위미리 4207-1번지,NaN,NaN,NaN,NaN,0.064933,남원읍,0.032467
3,bk호텔제주,숙박,9.0,16.0,15.0,11.0,15.0,13.0,#호텔 #숙소 #비즈니스센터 #컨시어지서비스 #엘리베이터,1.0,...,126.56433378,33.24099288,제주특별자치도 서귀포시 서귀동 777-1번지,제주특별자치도 서귀포시 서귀동 777-1번지,0.0,0.0,0.445723,0.581321,서귀동,0.010569
4,cs빌리지,숙박,23.0,28.0,33.0,22.0,16.0,22.0,#펜션 #숙소 #휴양펜션 #해안도로 #단체여행객 #독채 #바비큐 #온돌방 #가족 #...,0.0,...,126.36020727,33.47843562,제주특별자치도 제주시 애월읍 신엄리 2758번지,NaN,NaN,NaN,NaN,0.064933,애월읍,0.009276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,휘슬락호텔,숙박,6.0,11.0,10.0,17.0,16.0,15.0,#호텔 #숙소 #오션뷰호텔 #공공와이파이존 #수영장 #주차장 #양식레스토랑 #조식포함,2.0,...,126.52752517,33.51735064,제주특별자치도 제주시 건입동 1443-2번지,제주특별자치도 제주시 건입동 1443-2번지,0.0,0.0,2.225807,3.073150,건입동,0.008806
738,휴마루리조트,숙박,16.0,14.0,20.0,24.0,28.0,29.0,#휴식/치유 #숙소 #리조트 #휴양콘도 #공공와이파이존 #자연경관 #수영장 #환전 ...,1.0,...,126.54269631,33.26124030,제주특별자치도 서귀포시 서홍동 1777-3번지,제주특별자치도 서귀포시 서홍동 1777-3번지,0.0,0.0,0.063011,0.092380,서홍동,0.004619
739,흰고래게스트하우스,숙박,17.0,12.0,18.0,32.0,22.0,25.0,#게스트하우스 #숙소 #독채 #공공와이파이존 #가족 #돌담길,0.0,...,126.52369834,33.24393635,제주특별자치도 서귀포시 서호동 65-2번지,NaN,NaN,NaN,NaN,NaN,서호동,NaN
740,흰수염고래리조트,숙박,10.0,28.0,20.0,46.0,64.0,59.0,#휴식 #숙소 #리조트 #주차장 #공공와이파이존 #편의시설 #휴양콘도 #아동놀이방,1.0,...,126.38026116,33.47493633,제주특별자치도 제주시 애월읍 구엄리 941번지,제주특별자치도 제주시 애월읍 구엄리 941번지,0.0,0.0,0.244160,0.363100,애월읍,0.015129


In [133]:
lod_no_type.to_csv('data/숙박데이터_FINAL.csv', encoding='utf-8-sig', index=False)